In [1]:
import pandas as pd
import re
import requests
import xml.etree.ElementTree as ET
import time
import os
import csv

ModuleNotFoundError: No module named 'pandas'

In [ ]:
## !!! WE ARE NOT USING THIS !!!
def blind_extraction(text):
  for line in text:
    if line == "\n":
      newLine = True
      count +=1
      if count ==2:
          count = 0
          print(list_1)
          extracted_abstracts.append({
              'PMID': list_1[5],
              'doi': list_1[4],
              'title': list_1[1],
              'authors': list_1[2],
              'text': list_1[3],
          })
          list_1 = []
    else:
        lineData = line
        skip_append_flag = False

        # Extract DOI as a seperate item
        if(lineData.startswith("DOI:")):
          lineData = lineData.replace("DOI:", "")

        #Extract PMID as a seperate item
        if(lineData.startswith("PMID:")):
          lineData = lineData.replace("PMID:", "")
          lineData = lineData.replace("[Indexed for MEDLINE]", "")
          list_1.append(lineData.rstrip().lstrip())
          skip_append_flag = True

        if(lineData.startswith("Copyright")):
          skip_append_flag = True

        if(lineData.startswith("Author information:")):
          skip_append_flag = True
        # Extract publication name --- future work
        # Extract date published --- future work

        if(not skip_append_flag):
          if(newLine==False):
            lineData = list_1[-1] +" "+ lineData.rstrip().lstrip()
            list_1[-1] = lineData
          else:
            list_1.append(lineData.rstrip().lstrip())

        count = 0
        newLine=False

In [ ]:
abstract_data_dataframe = pd.DataFrame(columns=['PMID','doi', 'title', 'authors', 'text'])

In [ ]:
def checkCSVFile(file_name):
  if not os.path.isfile(file_name):
    # If the file doesn't exist, create it
    with open(file_name, 'w', newline='') as csvfile:
        # Create a CSV writer object
        csv_writer = csv.writer(csvfile)

        # Write headers or any initial data to the file
        headers = ['Column1', 'Column2', 'Column3']
        csv_writer.writerow(headers)

    print(f"CSV file '{file_name}' has been created.")
  else:
      print(f"CSV file '{file_name}' already exists.")

In [ ]:
global csv_abstract_filename
csv_abstract_filename = "/content/drive/MyDrive/NLPT(Abstract Data)/extracted_text_2.csv"

In [ ]:
checkCSVFile("/content/drive/MyDrive/NLPT(Abstract Data)/failedPmids.csv")

In [ ]:
checkCSVFile(csv_abstract_filename)

In [ ]:
def writeFailedPmid(pmid):
  failed_pmid_filename = "/content/drive/MyDrive/NLPT(Abstract Data)/failedPmids.csv"
  #writing to csv file
  with open(filename, 'a') as csvfile:
    #creating a csv writer object
    csvwriter = csv.writer(csvfile)

    # writing the data rows
    csvwriter.writerows([[pmid]])

In [ ]:
def writeDataToCSVFile(data):
  # name of csv file
  filename = csv_abstract_filename
  #writing to csv file
  with open(filename, 'a') as csvfile:
    #creating a csv writer object
    csvwriter = csv.writer(csvfile)

    # writing the data rows
    csvwriter.writerows([[data["pmid"], data["doi"], data["authors"], data["title"], data["text"]]])


In [ ]:
def recoverDataFromXML(XML_data):
  extracted_data = {
      "pmid":"",
      "doi":"",
      "authors":"",
      "title":"",
      "text":""
  }
  tree = ET.ElementTree(ET.fromstring(XML_data.replace('\n', '')))
  for elem in tree.iter():
    if(elem.tag == "ArticleTitle"):
      extracted_data["title"] = elem.text
    if(elem.tag == "ArticleId"):
      if(elem.attrib["IdType"] == "doi"):
        extracted_data["doi"] = elem.text
    if(elem.tag == "AbstractText"):
      extracted_data["text"] = elem.text
    if(elem.tag == "AuthorList"):
      author_list = []
      for child in elem:
        if(child.find("ForeName") != None and child.find("LastName") != None):
          author_name=getattr(child.find("ForeName"),"text")+" "+getattr(child.find("LastName"),"text")
          author_list.append(author_name)
      extracted_data["authors"] = author_list

  return extracted_data

In [ ]:
def getAbstractData(pmID, testResponseMode = False) :
  # api-endpoint
  URL = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id={pmID}&retmode=XML&rettype=abstract"

  # sending get request and saving the response as response object
  r = requests.get(url = URL)
  print(r)
  extractedText = None
  if(testResponseMode):
    print(r.text)
  elif (testResponseMode==False and r.status_code == 200):
    extractedText =  recoverDataFromXML(r.text)
    extractedText["pmid"] = pmID
    writeDataToCSVFile(extractedText)
  elif (testResponseMode==False and r.status_code != 200):
    writeFailedPmid(pmID)
  return extractedText

In [ ]:
getAbstractData(35801624, True)

In [ ]:

extracted_text = getAbstractData("36736583")


In [ ]:
file = open(f"/content/drive/MyDrive/NLPT(Abstract Data)/abstracts-original/abstract-intelligen-set (2).txt", 'r', encoding="utf-8")
text = file.readlines()
count = 0
pmid_extracted_counter = 0
api_rate_limiter = 0
pmid_val = ""
chunk_abstracts = []
newLine = True
for line in text:
  if line == "\n":
    newLine = True
    count +=1
    if count ==2:
      count = 0
      if(pmid_val != ""):
        pmid_extracted_counter += 1
        if(pmid_extracted_counter >= 1566):
          print(pmid_extracted_counter)
          extracted_text = getAbstractData(pmid_val)
          api_rate_limiter += 1
          if(api_rate_limiter == 3):
            time.sleep(1)
            api_rate_limiter = 0
        #abstract_data_dataframe = pd.concat([abstract_data_dataframe, pd.DataFrame(extracted_text)], axis=0, ignore_index=True)
      list_1 = []
  else:
    count = 0
    lineData = line
    #Extract PMID as a seperate item
    if(lineData.startswith("PMID:")):
      lineData = lineData.replace("PMID:", "")
      lineData = lineData.replace("[Indexed for MEDLINE]", "")
      pmid_val = lineData.rstrip().lstrip()







In [ ]:
len(chunk_abstracts)

In [ ]:
def getPMIDs ():
  chunk_abstracts = []
  for i in range(0,1):
      file = open(f"/content/drive/MyDrive/NLPT(Abstract Data)/abstracts-original/abstract-intelligen-set (1).txt", 'r', encoding="utf-8")
      text = file.readlines()
      count = 0
      pmid_val = ""
      chunk_abstracts = []
      newLine = True
      for line in text:
        if line == "\n":
          newLine = True
          count +=1
          if count ==2:
            count = 0
            if(pmid_val != ""):
              chunk_abstracts.append(pmid_val)
              #abstract_data_dataframe = pd.concat([abstract_data_dataframe, pd.DataFrame(extracted_text)], axis=0, ignore_index=True)
            list_1 = []
        else:
          count = 0
          lineData = line
          #Extract PMID as a seperate item
          if(lineData.startswith("PMID:")):
            lineData = lineData.replace("PMID:", "")
            lineData = lineData.replace("[Indexed for MEDLINE]", "")
            pmid_val = lineData.rstrip().lstrip()
  return chunk_abstracts







In [ ]:
chunk_abstracts = getPMIDs()

In [ ]:
len(chunk_abstracts)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/NLPT(Abstract Data)/extracted_text.csv')

In [ ]:
df.columns = ['PMID','doi', 'title', 'authors', 'text']

In [ ]:
df.head()

In [ ]:
read_pmids = df["PMID"].astype(str).tolist()

In [ ]:
read_pmids[1]

In [ ]:
def find_missing_elements(arr1, arr2):
    missing_elements = []
    # Convert arrays to sets for faster comparison
    set1 = set(arr1)
    set2 = set(arr2)

    # Find elements in arr1 that are not in arr2
    missing_elements = list(set1 - set2)

    return missing_elements

In [ ]:
missed_pmids = find_missing_elements(chunk_abstracts,read_pmids)

In [ ]:
len(missed_pmids)

In [ ]:
def addMissedPmidsToCSV(missed_pmids):
  api_rate_limiter = 0
  for i in range(0, len(missed_pmids)):
    print(i)
    extracted_text = getAbstractData(missed_pmids[i])
    api_rate_limiter += 1
    if(api_rate_limiter == 3):
      time.sleep(1)
      api_rate_limiter = 0

In [ ]:
addMissedPmidsToCSV(missed_pmids)